# ※このプログラムは良い精度が出なかった失敗作です

より良いプログラムが作成できるまでの間、一時的にアップしています。

#◯タイトル：Tabular Playground Series - Mar 2022

・こちらのコードは、データ分析コンペティションkaggleで毎月開催されているTabular Playground Seriesの2022年3月版についてのものです。

◯目的：アメリカの大都市での12時間の渋滞状況を予測する

◯データ(データの例）
*   row_id - - - id(0)
*   time - - - 時刻(1991-04-01 00:00:00)
*   x      - - - 横方向の座標(0)
*   y     - - - 縦方向の座標(0)
*  direction - - - 方角(EB)
*  congestion - - - 混み具合(70)

◯テストデータから予測したcongestionがtestデータのcongestionに近くなるように学習を行う

◯用いるモデル：PyTorchを用いたMLPネットワーク

◯工夫した点


*   自分の仮説に基づいて特徴量の生成を行ったこと
*   データをtensorに変換する過程



◯課題


1.   Tensorに変換するために、one-hot-encodingをしてしまった為、PyTorchの強みを活かせなかった



1.モジュールのImport

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from sklearn.compose  import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

2.データの操作

In [ ]:
# データの読み込み

train = pd.read_csv("train.csv",index_col="row_id", parse_dates=["time"])
test = pd.read_csv("test.csv", index_col="row_id", parse_dates=["time"])

train.head()

,time,x,y,direction,congestion
row_id,,,,,
0,1991-04-01,0,0,EB,70
1,1991-04-01,0,0,NB,49
2,1991-04-01,0,0,SB,24
3,1991-04-01,0,1,EB,18
4,1991-04-01,0,1,NB,60


In [ ]:
# データの全体像を把握する

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848835 entries, 0 to 848834
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        848835 non-null  datetime64[ns]
 1   x           848835 non-null  int64         
 2   y           848835 non-null  int64         
 3   direction   848835 non-null  object        
 4   congestion  848835 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 38.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 848835 to 851174
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       2340 non-null   datetime64[ns]
 1   x          2340 non-null   int64         
 2   y          2340 non-null   int64         
 3   direction  2340 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 91.4+ KB


In [ ]:
# trainのcongestionに欠損値が確認できたので穴埋めを行う
# 欠損データ周辺の時間帯におけるcongestionと似通っているという仮説から、直前のデータで穴埋めをする

train["congestion"] = train["congestion"].fillna(method="ffill")
train.isnull().all()

time          False
x             False
y             False
direction     False
congestion    False
dtype: bool

In [ ]:
# datetime型の性質を利用して新たな特徴量を生成

train["year"] = train["time"].dt.year
train["month"] = train["time"].dt.month
train["day"] = train["time"].dt.day
train["hour"] = train["time"].dt.hour
train["minute"] = train["time"].dt.minute
train["day_of_week"] = train["time"].dt.weekday

train.sample(n=5).sort_index()

,time,x,y,direction,congestion,year,month,day,hour,minute,day_of_week
row_id,,,,,,,,,,,
410053,1991-06-27 21:20:00,1,3,EB,44,1991,6,27,21,20,3
507264,1991-07-18 16:00:00,0,1,NB,77,1991,7,18,16,0,3
608783,1991-08-09 10:00:00,2,2,WB,64,1991,8,9,10,0,4
731537,1991-09-04 22:20:00,1,2,EB,59,1991,9,4,22,20,2
798704,1991-09-19 15:00:00,2,1,SW,47,1991,9,19,15,0,3


In [ ]:
# 給料日が来た後に外出する人が増えるという仮説から、月初か月終わりかを判定する特徴量を追加
train["is_month_start"] = train["time"].dt.is_month_start.astype(int)
train["is_month_end"] = train["time"].dt.is_month_end.astype(int)

train.sample(n=5).sort_index()

,time,x,y,direction,congestion,year,month,day,hour,minute,day_of_week,is_month_start,is_month_end
row_id,,,,,,,,,,,,,
2665,1991-04-01 13:40:00,0,0,EB,65,1991,4,1,13,40,0,1,0
105174,1991-04-23 11:40:00,0,1,NB,74,1991,4,23,11,40,1,0,0
261843,1991-05-27 03:40:00,1,1,EB,28,1991,5,27,3,40,0,0,0
507080,1991-07-18 15:00:00,0,3,SW,36,1991,7,18,15,0,3,0,0
583080,1991-08-03 21:40:00,1,2,SB,57,1991,8,3,21,40,5,0,0


In [ ]:
# 時間と分は独立して判断するものではないので、合わせる
train["hour+minute"] = train["hour"] * 60 + train["minute"]

# 平日と休日では混雑状況が異なるという仮説から、休日か平日かを判定する特徴量を追加
train["is_weekend"] = (train["day_of_week"] > 4).astype("int")

# 午前と午後かによって交通量が異なるという仮説から、午前か午後かを判定する特徴を追加
train["is_afternoon"] = (train["hour"] > 12).astype("int")

train = train.drop(["time"], axis=1)
train.sample(n=5).sort_index()

,x,y,direction,congestion,year,month,day,hour,minute,day_of_week,is_month_start,is_month_end,hour+minute,is_weekend,is_afternoon
row_id,,,,,,,,,,,,,,,
151510,2,3,NB,57,1991,5,3,11,20,4,0,0,680,0,0
363410,2,3,NB,65,1991,6,17,22,0,0,0,0,1320,0,1
570782,1,0,EB,46,1991,8,1,6,40,3,1,0,400,0,0
587010,2,3,NB,57,1991,8,4,17,40,6,0,0,1060,1,1
641088,2,2,WB,77,1991,8,16,7,40,4,0,0,460,0,0


In [ ]:
# 同様の処理をtestにも行う

test["year"] = test["time"].dt.year
test["month"] = test["time"].dt.month
test["day"] = test["time"].dt.day
test["hour"] = test["time"].dt.hour
test["minute"] = test["time"].dt.minute
test["day_of_week"] = test["time"].dt.weekday
test["is_month_start"] = test["time"].dt.is_month_start.astype(int)
test["is_month_end"] = test["time"].dt.is_month_end.astype(int)
test["hour+minute"] = test["hour"]*60 + test["minute"]
test['is_weekend'] = (test['day_of_week'] > 4).astype('int')
test['is_afternoon'] = (test['hour'] > 12).astype('int')
test = test.drop(["time"], axis=1)
test.sample(n=5).sort_index()

,x,y,direction,year,month,day,hour,minute,day_of_week,is_month_start,is_month_end,hour+minute,is_weekend,is_afternoon
row_id,,,,,,,,,,,,,,
849371,0,3,WB,1991,9,30,14,40,0,0,1,880,0,1
849446,1,1,WB,1991,9,30,15,0,0,0,1,900,0,1
850008,0,1,EB,1991,9,30,18,0,0,0,1,1080,0,1
850669,0,3,SB,1991,9,30,21,20,0,0,1,1280,0,1
850961,2,1,NW,1991,9,30,22,40,0,0,1,1360,0,1


In [ ]:
target = train.congestion
train = train.drop(columns="congestion")

train.head()

,x,y,direction,year,month,day,hour,minute,day_of_week,is_month_start,is_month_end,hour+minute,is_weekend,is_afternoon
row_id,,,,,,,,,,,,,,
0,0,0,EB,1991,4,1,0,0,0,1,0,0,0,0
1,0,0,NB,1991,4,1,0,0,0,1,0,0,0,0
2,0,0,SB,1991,4,1,0,0,0,1,0,0,0,0
3,0,1,EB,1991,4,1,0,0,0,1,0,0,0,0
4,0,1,NB,1991,4,1,0,0,0,1,0,0,0,0


In [ ]:
# one-hot-encodingを行う
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train.head()

,x,y,year,month,day,hour,minute,day_of_week,is_month_start,is_month_end,...,is_weekend,is_afternoon,direction_EB,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB
row_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1991,4,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,1991,4,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,1991,4,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,1991,4,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,1991,4,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
num_feature = train.shape[1]
num_feature

21

In [ ]:
# Tensorに変換する

train = torch.tensor(train.values)
test = torch.tensor(test.values)
target = target.values.reshape(-1,1)
target = torch.tensor(target)

In [ ]:
# GPUが使える環境かを確認

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
train = train.float().to(device)
test = test.float().to(device)
target = target.float().to(device)

In [ ]:
# Dataloaderの定義

train, val, train_target, val_target = train_test_split(train, target)
train_dataset = torch.utils.data.TensorDataset(train, train_target)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val, val_target)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)

In [ ]:
# Dataloaderの確認

for data, target in train_dataloader:
  print(data.size())
  print(data[0].size())
  print(target.size())
  print(data.type())
  break

torch.Size([128, 21])
torch.Size([21])
torch.Size([128, 1])
torch.cuda.FloatTensor


3.モデルの作成

In [ ]:
# MLPネットワークを定義する
# 4層から成る多層パーセプトロンにドロップアウトと活性化関数としてrelu関数を用いたモデル

class MLPNet(nn.Module):
  def __init__(self):
    super(MLPNet, self).__init__()
    self.fc1 = nn.Linear(num_feature, 150)
    self.fc2 = nn.Linear(150, 100)
    self.fc3 = nn.Linear(100, 50)
    self.fc4 = nn.Linear(50, 1)
    self.dropout1 = nn.Dropout2d(0.2)
    self.dropout2 = nn.Dropout2d(0.2)
    self.dropout3 = nn.Dropout2d(0.2)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout1(x)
    x = F.relu(self.fc2(x))
    x = self.dropout2(x)
    x = F.relu(self.fc3(x))
    x = self.dropout3(x)
    x = F.relu(self.fc4(x))

    return x

net = MLPNet().to(device)

In [ ]:
# 損失関数と最適化手法の選択

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

4.学習

In [ ]:
# 1000エポック分学習をする

num_epochs = 1000

# 最後にlossのグラフをプロットするためのリストを作成
train_loss_list = []
val_loss_list = []

In [ ]:
# 学習

for epoch in range(num_epochs):
  # エポックごとに初期化
  train_loss = 0
  val_loss = 0
  
  # ========================= train =============================

  # trainモードに切り替え
  net.train()
  # ミニバッチに分割して読み込む
  for i, (data, target) in enumerate(train_dataloader):
    # 勾配をリセット
    optimizer.zero_grad()
    # 順伝播
    outputs = net(data)
    # 誤差の計算
    loss = criterion(outputs, target)
    # 誤差をミニバッチ分足し合わせる
    train_loss += loss.item()
    # 逆伝播
    loss.backward()
    # 重みの更新
    optimizer.step()

  # 平均誤差を計算
  avg_train_loss = train_loss / len(train_dataloader.dataset)

  # ========================= val ===============================

  # valモードに切り替え
  net.eval()

  with torch.no_grad():
    for data,  target in val_dataloader:
      # 予測
      pred = net(data)
      # 誤差を計算
      loss = criterion(pred, target)
      # 誤差をミニバッチ分足し合わせる
      val_loss += loss.item()
      
  avg_val_loss = val_loss / len(val_dataloader.dataset)

  # trainデータの誤差とvalデータの誤差をログで出力する
  print ('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}' 
                   .format(epoch+1, num_epochs, loss=avg_train_loss, val_loss=avg_val_loss))
  # グラフをプロットするためにリストに格納
  train_loss_list.append(avg_train_loss)
  val_loss_list.append(avg_val_loss)

Epoch [1/1000], Loss: 284935854781330.5625, val_loss: 20.0375
Epoch [2/1000], Loss: 20.0782, val_loss: 20.0374
Epoch [3/1000], Loss: 20.0782, val_loss: 20.0374


KeyboardInterrupt: ignored